In [3]:
:set -XNoImplicitPrelude
:set -XTupleSections
import Papa
import qualified Numeric.LinearAlgebra as HM
import qualified Numeric.LinearAlgebra.Data as HM
-- import Algebra.Graph.AdjacencyMap
import qualified Algebra.Graph.Labelled.AdjacencyMap as GL
import qualified Algebra.Graph.AdjacencyMap as G
import Algebra.Graph.Label
import qualified Data.Map as M (fromList, Map(..))
import Data.Tuple (swap)
import Data.Tuple.Extra ((***))
import Foreign.Storable (Storable(..))
import Prelude (undefined, enumFromTo)

In [50]:
prob :: Num t => [[t]]
prob = [[-1, 1, -1, 1, -1],
        [-1, -1, -1, 2, -1],
        [2, -1, -1, -1, -1],
        [-1, 3, -1, -1, 2],
        [-1, 1, -1, -1, -1]]
        
m :: HM.Matrix Double
m = HM.fromLists prob

data Vert a = Row a | Col a deriving (Ord, Show, Eq)

withIndex :: [a] -> [(Int, a)]
withIndex xs = zip [0 .. length xs] xs

onlyVals :: (Num a, Ord a) => [(Int, a)] -> [(Int, a)]
onlyVals = Papa.filter ( (> 0) . snd)

row :: (a -> b) -> (Int, a) -> (Vert Int, b)
row f = Row *** f

col :: (a -> b) -> (Int, a) -> (Vert Int, b)
col f = Col *** f

rowColGraph :: [HM.Vector Double] -> GL.AdjacencyMap (Distance (Sum Double)) (Vert Int)
rowColGraph =  GL.fromAdjacencyMaps . 
                ((row (M.fromList . (col (distance . return . return) <$>) . onlyVals . withIndex . HM.toList)) <$>) .
                withIndex

rowGraph :: [HM.Vector Double] -> GL.AdjacencyMap (Distance (Sum Double)) (Vert Int)
rowGraph xs = GL.edges $ 
                ((distance 0,,) <$> fst <*> snd) <$>
                G.edgeList (G.path $ (Row <$> [0 .. length xs - 1]))

adjGraph :: HM.Matrix Double -> GL.AdjacencyMap (Distance (Sum Double)) (Vert Int)
adjGraph mat =  let rs = HM.toRows mat 
                    a = rowColGraph rs
                    b = rowGraph rs
                in
                    GL.overlay b a

In [58]:
import Algebra.Graph.AdjacencyMap.Algorithm

import Algebra.Graph.ToGraph
import Algebra.Graph.Export.Dot

a = adjGraph m
tra = GL.transitiveClosure a
putStrLn $ exportViaShow a
putStrLn $ exportViaShow tra
-- GL.adjacencyMap a
-- putStrLn $ exportViaShow $ GL.transitiveClosure a
-- GL.adjacencyMap tra


digraph 
{
  "Row 0"
  "Row 1"
  "Row 2"
  "Row 3"
  "Row 4"
  "Col 0"
  "Col 1"
  "Col 3"
  "Col 4"
  "Row 0" -> "Row 1"
  "Row 0" -> "Col 1"
  "Row 0" -> "Col 3"
  "Row 1" -> "Row 2"
  "Row 1" -> "Col 3"
  "Row 2" -> "Row 3"
  "Row 2" -> "Col 0"
  "Row 3" -> "Row 4"
  "Row 3" -> "Col 1"
  "Row 3" -> "Col 4"
  "Row 4" -> "Col 1"
}

digraph 
{
  "Row 0"
  "Row 1"
  "Row 2"
  "Row 3"
  "Row 4"
  "Col 0"
  "Col 1"
  "Col 3"
  "Col 4"
  "Row 0" -> "Row 1"
  "Row 0" -> "Row 2"
  "Row 0" -> "Row 3"
  "Row 0" -> "Row 4"
  "Row 0" -> "Col 0"
  "Row 0" -> "Col 1"
  "Row 0" -> "Col 3"
  "Row 0" -> "Col 4"
  "Row 1" -> "Row 2"
  "Row 1" -> "Row 3"
  "Row 1" -> "Row 4"
  "Row 1" -> "Col 0"
  "Row 1" -> "Col 1"
  "Row 1" -> "Col 3"
  "Row 1" -> "Col 4"
  "Row 2" -> "Row 3"
  "Row 2" -> "Row 4"
  "Row 2" -> "Col 0"
  "Row 2" -> "Col 1"
  "Row 2" -> "Col 4"
  "Row 3" -> "Row 4"
  "Row 3" -> "Col 1"
  "Row 3" -> "Col 4"
  "Row 4" -> "Col 1"
}

In [65]:
import Algebra.Graph
import Data.Tree

scc $ toAdjacencyMap $ tree (Node 1 [Node 2 [], Node 3 [Node 4 [], Node 5 []]]) 

edges [(vertex 1,vertex 2),(vertex 1,vertex 3),(vertex 3,vertex 4),(vertex 3,vertex 5)]